# Esercitazione 1 - Lexical Overlap

Studenti:

- Brunello Matteo (mat. 858867)
- Caresio Lorenzo (mat. 836021)

*Consegna*: misurazione dell’overlap lessicale tra una serie di definizioni per concetti generici/specifici e concreti/astratti. Partendo dai dati sulle definizioni (presente nella cartella "dati" su Moodle), si richiede di calcolare la similarità $2$-a-$2$ tra le definizioni (ad es. usando la cardinalità dell'intersezione dei lemmi normalizzata sulla lunghezza minima delle definizioni), aggregando (ed effettuando la media degli score di similarità) sulle due dimensioni (concretezza / specificità).

## Caricamento e preprocessing del dataset

Da una prima valutazione del dataset fornito in formato `tsv` ne emerge che le varie colonne si riferiscono ai termini, mentre le righe si riferiscono alle definizioni associati ad essi. Per ottenere un formato atto a calcolare la similarità, ogni definizione viene pre-processata trasformandola in tokens (che in questo caso sono *parole*) rimuovendo eventualmente la punteggiatura. Una volta ottenuta una lista di token rappresentanti la definizione, ogni token viene successivamente lemmatizzato utilizzando WordNet.

Il risultato del caricamento del dataset è un dizionario che ha come chiavi i termini e come valore una lista di definizioni tokenizzate.

In [ ]:
# Open the 'inspect element' tab while browsing into your moodle session, then paste the "MoodleSession" field value in the storage/cookies tab
moodle_session_cookie = 'b39kmpptestnpipi4a4v8vcslq'

!curl --cookie 'MoodleSession={moodle_session_cookie}' "https://informatica.i-learn.unito.it/pluginfile.php/366022/mod_folder/content/0/TLN-definitions-23.tsv?forcedownload=1" -o definitions.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8514  100  8514    0     0   9355      0 --:--:-- --:--:-- --:--:--  9356


In [ ]:
import csv
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
import string
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Define a set of tokens that must be filtered out
stop = set(stopwords.words('english') + list(string.punctuation))

# Define the lemmatizer
lemmatizer = WordNetLemmatizer()

# Tokenize a sentence, filtering out all stopwords and punctuations in the process.
# The function then "tries" to lemmatize each token using Wordnet.
def tokenize(sentence):
  return [lemmatizer.lemmatize(i) for i in word_tokenize(sentence.lower()) if i not in stop]

# Load definitions excluding the line number,
# returning the result as a dictionary
def load_definitions(path: str):
  result = {}
  with open(path, 'r') as f:
    content = list(csv.reader(f, delimiter = '\t'))
    for i, word in enumerate(content[0][1:]):
      result[word] = [tokenize(definition[i+1]) for definition in content[1:]]
    return result

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Calcolo overlap lessicale
Come suggerito dalla consegna, definiamo la similarità come overlap lessicale. Successivamente si calcola l'overlap medio, considerando le varie definizioni (precedentemente tokenizzate) a due a due.

In [ ]:
from itertools import combinations
from statistics import mean

# Compute the similarity between two definitions
def similarity(d1, d2):
  d1, d2 = set(d1), set(d2)
  return len(d1 & d2) / min(len(d1), len(d2))

# Compute the average pairwise similarity between definitions for each word
def compute_overlaps(defs):
  for word in defs.keys():
    defs[word] = mean([similarity(d1, d2) for d1, d2 in combinations(defs[word], 2)])
  return defs

Infine, si va a visualizzare i risultati ottenuti in un formato tabellare.


In [ ]:
def pretty_print(overlaps):
    printable_pairs = [f'{w}: {m:0.2f}' for w, m in overlaps.items()]
    print('+-----------+---------------------------+--------------------------+')
    print(f'|{"":10s} | {"Abstract":25s} | {"Concrete":25s}|')
    print('+-----------+---------------------------+--------------------------+')
    print(f'|{"Generic":10s} | {str(printable_pairs[2]):25s} | {str(printable_pairs[0]):25s}|')
    print(f'|{"Specific":10s} | {str(printable_pairs[3]):25s} | {str(printable_pairs[1]):25s}|')
    print('+-----------+---------------------------+--------------------------+')

In [ ]:
# Execute all the steps defined earlier
path = 'definitions.tsv'
definitions = load_definitions(path)
overlaps = compute_overlaps(definitions)
pretty_print(overlaps)

+-----------+---------------------------+--------------------------+
|           | Abstract                  | Concrete                 |
+-----------+---------------------------+--------------------------+
|Generic    | pain: 0.20                | door: 0.19               |
|Specific   | blurriness: 0.07          | ladybug: 0.56            |
+-----------+---------------------------+--------------------------+


Dai risultati emerge che per un concetto concreto e specifico come *ladybug* le definizioni sono molto simili tra loro ($\approx 56\%$), mentre per altri concetti i risultati sono meno significativi, evidente nel concetto di *blurriness*, specifico e astratto, dove l'overlap medio tra le varie definizioni è molto basso (suggerendo la difficoltà a raggiungere un consenso generale in termini di definizioni di questo concetto).